In [1]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


# Setup

In [2]:
x_train = np.load("../input/ndscloaded/x_train.npy")
x_test = np.load("../input/ndscloaded/x_test.npy")
y_train = np.load("../input/ndscloaded/y_train.npy")
features = np.load("../input/ndscloaded/features.npy")
test_features = np.load("../input/ndscloaded/test_features.npy")
word_index = np.load("../input/ndscloaded/word_index.npy").item()
embedding_matrix = np.load("../input/ndscloaded/embedding_matrix.npy")

In [3]:
print(y_train)

[ 2 15 18 ... 25 25  7]


In [4]:
print(np.shape(embedding_matrix[25]))

(300,)


In [11]:
batch_size = 128

indices = np.arange(len(x_train)-1)
np.random.shuffle(indices)
i = 0
x_train_batch_tokens = np.take(x_train, indices[i*batch_size:(i+1)*batch_size], axis = 0) 
x_train_batch = [[embedding_matrix[token] for token in title] for title in x_train_batch_tokens]
y_train_batch_targets = np.take(y_train, indices[i*batch_size:(i+1)*batch_size])
y_train_batch = np.zeros((batch_size, 58))
y_train_batch[np.arange(batch_size), y_train_batch_targets] = 1

print(indices)
print(x_train_batch_tokens)
print(np.shape(x_train_batch))
print(y_train_batch_targets)
print(y_train_batch)

print(type(indices))
print(type(x_train_batch_tokens))
print(type(np.shape(x_train_batch[:1])))
print(type(x_train_batch))
print(type(y_train_batch_targets))
print(type(y_train_batch))

[497043 363007 216575 ...  92956  39481 295983]
[[   0    0    0 ...   45  269   27]
 [   0    0    0 ...   62   63  371]
 [   0    0    0 ...   49  159   97]
 ...
 [   0    0    0 ...  175    3 1119]
 [   0    0    0 ...   96    8  222]
 [   0    0    0 ...  472    6    4]]
(128, 70, 300)
[29 35 32  5 18 12 18 31 29 26 38 20 35 22  3  3 20 24 18  1 31  3 16  1
 24  5 34 18  7  4 34  4  5 21 34 35 22  6 22  5 29 27 18  9  3 25 12 26
 18 38 41 27 22  3  5 41 35 25  4 13  5 32 11 18 12 43 28  9 27 32  4 28
 34  7 41 34  2 25  4  3 35  4 35  4 41 22  7 25 42 25 18 20  3  3  3 45
 22  5 37 32  3 28  5 21  5 22 32  3 18 41  3 31 25 38 26  3  5 20 25 21
 26  3  5 25  4 18 31 21]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'tuple'>
<class 'list'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [12]:
# Data providers
batch_size = 128

indices = np.arange(len(x_train)-1)
# print(len(x_train))
# print(np.random.shuffle())

def batch_gen():
    n_batches = math.ceil(len(x_train) / batch_size)
    while True: 
        np.random.shuffle(indices)  # Shuffle the data.
#         print(indices)
        for i in range(n_batches):
#             texts = train_df.iloc[i*batch_size:(i+1)*batch_size, 1]
#             text_arr = np.array([text_to_array(text) for text in texts])
            
            x_train_batch_tokens = np.take(x_train, indices[i*batch_size:(i+1)*batch_size], axis = 0) 
            x_train_batch = [[embedding_matrix[token] for token in title] for title in x_train_batch_tokens]
            y_train_batch_targets = np.take(y_train, indices[i*batch_size:(i+1)*batch_size])
            y_train_batch = np.zeros((batch_size, 58))
            y_train_batch[np.arange(batch_size), y_train_batch_targets] = 1

            yield (np.array(x_train_batch), y_train_batch)

# Training

In [13]:
from keras.models import Sequential
from keras.layers import CuDNNLSTM, Dense, Bidirectional, Activation

In [16]:
model = Sequential()
model.add(Bidirectional(CuDNNLSTM(64, return_sequences=True),
                        input_shape=(70, 300)))
model.add(Bidirectional(CuDNNLSTM(64)))
model.add(Dense(58))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [17]:
mg = batch_gen()
model.fit_generator(mg, epochs=1,
                    steps_per_epoch=1000,
                    verbose=True)

Epoch 1/1


InvalidArgumentError: No OpKernel was registered to support Op 'CudnnRNN' with these attrs.  Registered devices: [CPU,XLA_CPU], Registered kernels:
  <no registered kernels>

	 [[node bidirectional_1/CudnnRNN (defined at /usr/local/lib/python3.5/dist-packages/tensorflow/contrib/cudnn_rnn/python/ops/cudnn_rnn_ops.py:922)  = CudnnRNN[T=DT_FLOAT, direction="unidirectional", dropout=0, input_mode="linear_input", is_training=true, rnn_mode="lstm", seed=87654321, seed2=0](bidirectional_1/transpose, bidirectional_1/ExpandDims_1, bidirectional_1/ExpandDims_2, bidirectional_1/concat)]]

Caused by op 'bidirectional_1/CudnnRNN', defined at:
  File "/usr/lib/python3.5/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.5/dist-packages/traitlet

# Inference

In [ ]:
# prediction part
batch_size = 256
def batch_gen(test_df):
    n_batches = math.ceil(len(test_df) / batch_size)
    for i in range(n_batches):
        texts = test_df.iloc[i*batch_size:(i+1)*batch_size, 1]
        text_arr = np.array([text_to_array(text) for text in texts])
        yield text_arr

test_df = pd.read_csv("../input/ndsc-beginner/test.csv")

all_preds = []
for x in tqdm(batch_gen(test_df)):
    all_preds.extend(model.predict(x).flatten())

In [ ]:
print(np.shape(all_preds)
y_te = [np.argmax(pred) for pred in all_preds]

submit_df = pd.DataFrame({"itemid": test_df["itemid"], "Category": y_te})
submit_df.to_csv("submission.csv", index=False)